In [2]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-07-26 04:11:11--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.114.128, 142.250.1.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.114.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.02s   

2023-07-26 04:11:11 (153 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [6]:
import tensorflow as tf
import string
import re
import random
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# 데이터 확인하기
text_file = 'spa-eng/spa.txt'
with open(text_file) as f:
	lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
	english, spanish = line.split("\t")
	spanish = '[start] ' + spanish + ' [end]'
	text_pairs.append((english, spanish))

# text_pairs 내용

print(random.choice(text_pairs))

('This wine tastes great.', '[start] Este vino sabe estupendo. [end]')


In [8]:
# 데이터 섞은 뒤 훈련/검증/테스트 나누기
import random

random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples : ]

In [45]:
# 11-26. 영어 & 스페인어 텍스트 쌍 벡터화
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    # `[, ]` 문자 유지, ¿ 문자를 포함한 strings.punctuation에 있는 다른 문자 삭제
	lower_case = tf.strings.lower(input_string)
	return tf.strings.regex_replace(
	lower_case, f"[{re.escape(strip_chars)}]", "")

voca_size = 15000
sequence_length = 20

# 영어
source_vectorization = layers.TextVectorization(
												max_tokens = voca_size,
												output_mode = 'int',
												output_sequence_length = sequence_length
)

# 스페인어
target_vectorization = layers.TextVectorization(
												max_tokens = voca_size,
												output_mode = 'int',
												output_sequence_length = sequence_length + 1, # 훈련 중 한 스텝 앞선 문장이 필요하기 때문에 토큰 1개 추가
												standardize = custom_standardization
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [46]:
# 데이터 -> 파이프라인 변환
batch_size = 64

def format_dataset(eng, spa):
	eng = source_vectorization(eng)
	spa = target_vectorization(spa)
	return ({
		'english' : eng,
		'spanish' : spa[:, :-1] # 입력 스페인어 문장은 마지막 토큰 포함 X라서 입출력 길이 동일
	}, spa[:, 1:]) # 타깃 스페인어 문장

def make_dataset(pairs):
	eng_texts, spa_texts = zip(*pairs)
	eng_texts = list(eng_texts)
	spa_texts = list(spa_texts)
	dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
	dataset = dataset.batch(batch_size)
	dataset = dataset.map(format_dataset, num_parallel_calls = 4)
	return dataset.shuffle(2048).prefetch(16).cache() # 전처리 속도 높이기 위한 캐싱

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

# 크기 확인
for inputs, targets in train_ds.take(1):
	print(f"inputs['english'].shape : {inputs['english'].shape}")
	print(f"inputs['spanish'].shape : {inputs['spanish'].shape}")
	print(f"targets.shape : {targets.shape}")
	print(inputs['english'][0])

inputs['english'].shape : (64, 20)
inputs['spanish'].shape : (64, 20)
targets.shape : (64, 20)
tf.Tensor(
[   6   14 1312   10  813    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0], shape=(20,), dtype=int64)


In [47]:
class PositionalEmbedding(layers.Layer):

  # 위치 임베딩 : 시퀀스 길이를 미리 알아야 한다는 게 단점임
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(
        input_dim = input_dim, output_dim = output_dim
    )
    self.position_embeddings = layers.Embedding(
        input_dim = sequence_length, output_dim = output_dim
    )
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start = 0, limit = length, delta = 1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  # 입력 0 패딩을 무시하는 마스킹 생성.
  # 프레임워크에 의해 자동으로 호출되며, 마스킹은 다음 층으로 전달된다.
  def compute_mask(self, inputs, mask = None):
    return tf.math.not_equal(inputs, 0)

  def get_config(self):
    config = super().get_config()
    config.update({
        "output_dim" : self.output_dim,
        "sequence_length" : self.sequence_length,
        "input_dim" : self.input_dim
    })
    return config


In [48]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim # 입력 토큰 벡터 크기
    self.dense_dim = dense_dim # 내부 밀집 층 크기
    self.num_heads = num_heads # 어텐션 헤드 개수
    self.attention = layers.MultiHeadAttention(
        num_heads = num_heads, key_dim = embed_dim
    )
    self.dense_proj = keras.Sequential(
        [
            layers.Dense(dense_dim, activation = 'relu'),
            layers.Dense(embed_dim),
        ]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  # 연산 수행
  def call(self, inputs, mask = None):

    # Embedding층의 마스크는 2D이나, 어텐션층은 3D나 4D 기대
    if mask is not None:
      mask = mask[:, tf.newaxis, :]

    attention_output = self.attention(inputs, inputs, attention_mask = mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  # 모델 저장을 위한 직렬화 구현
  def get_config(self):
    config = super().get_config()
    config.update({
        'embed_dim' : self.embed_dim,
        'num_heads' : self.num_heads,
        'dense_dim' : self.dense_dim
    })
    return config

In [49]:
class TransformerDecoder(layers.Layer):
	def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
		super().__init__(**kwargs)
		self.embed_dim = embed_dim
		self.dense_dim = dense_dim
		self.num_heads = num_heads
		self.attention_1 = layers.MultiHeadAttention(num_heads = num_heads,
		key_dim = embed_dim)
		self.attention_2 = layers.MultiHeadAttention(num_heads = num_heads,
		key_dim = embed_dim)
		self.attention_3 = layers.MultiHeadAttention(num_heads = num_heads,
		key_dim = embed_dim)
		self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation = 'relu'),
		layers.Dense(embed_dim),])
		self.layernorm_1 = layers.LayerNormalization()
		self.layernorm_2 = layers.LayerNormalization()
		self.layernorm_3 = layers.LayerNormalization()
		self.supports_masking = True # 입력 마스킹을 출력으로 전달하게 함
		# layer.compute_mask()는 위 속성이 False이면 에러를 반환한다.
	def get_config(self):
		config = super().get_config()
		config.update({
		'embed_dim' : self.embed_dim,
		'num_heads' : self.num_heads,
		'dense_dim' : self.dense_dim
		  })
		return config

	# 코잘 마스킹 : 미래 타임스텝의 데이터를 사용하지 못하게 한다
	def get_casual_attention_mask(self, inputs):
		input_shape= tf.shape(inputs)
		batch_size, sequence_length = input_shape[0], input_shape[1]
		i = tf.range(sequence_length)[:, tf.newaxis]
		j = tf.range(sequence_length)
		mask = tf.cast(i >= j, dtype = 'int32') #
		mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))

		mult = tf.concat([tf.expand_dims(batch_size, -1),
						  tf.constant([1, 1], dtype = tf.int32)], axis = 0)
		return tf.tile(mask, mult)

	def call(self, inputs, encoder_outputs, mask = None):
		casual_mask = self.get_casual_attention_mask(inputs)

		if mask is not None:
			padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype = 'int32')
			padding_mask = tf.minimum(padding_mask, casual_mask)

		attention_output_1 = self.attention_1(
			query = inputs,
			value = inputs,
			key = inputs,
			attention_mask = casual_mask
		)
		attention_output_1 = self.layernorm_1(inputs + attention_output_1)

		attention_output_2 = self.attention_2(
			query = attention_output_1,
			value = encoder_outputs,
			key = encoder_outputs,
			attention_mask = padding_mask, # 합친 마스킹을 소스 + 타깃 시퀀스를 연결시키는 2번째 어텐션 층에 전달
		)
		attention_output_2 = self.layernorm_2(
			attention_output_1 + attention_output_2
		)

		proj_output = self.dense_proj(attention_output_2)
		return self.layernorm_3(attention_output_2 + proj_output)

In [50]:
# 엔드 투 엔드 트랜스포머
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape = (None,), dtype = 'int64', name = 'english')
x = PositionalEmbedding(sequence_length, voca_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape = (None, ), dtype = 'int64', name = 'spanish')
x = PositionalEmbedding(sequence_length, voca_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(voca_size, activation = 'softmax')(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.compile(
					 optimizer = 'rmsprop',
					 loss = 'sparse_categorical_crossentropy',
					 metrics = ['accuracy']
)
transformer.fit(train_ds, epochs = 5, validation_data = val_ds)

Epoch 1/5
1302/1302 [==============================] - 109s 77ms/step - loss: 3.7898 - accuracy: 0.4411 - val_loss: 2.9112 - val_accuracy: 0.5309
Epoch 2/5
1302/1302 [==============================] - 86s 66ms/step - loss: 2.8509 - accuracy: 0.5506 - val_loss: 2.5258 - val_accuracy: 0.5891
Epoch 3/5
1302/1302 [==============================] - 85s 65ms/step - loss: 2.5564 - accuracy: 0.5936 - val_loss: 2.3801 - val_accuracy: 0.6122
Epoch 4/5
1302/1302 [==============================] - 85s 66ms/step - loss: 2.3929 - accuracy: 0.6197 - val_loss: 2.3272 - val_accuracy: 0.6241
Epoch 5/5
1302/1302 [==============================] - 85s 66ms/step - loss: 2.2883 - accuracy: 0.6381 - val_loss: 2.2975 - val_accuracy: 0.6338


In [51]:
transformer.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_11 (Posit  (None, None, 256)   3845120     ['english[0][0]']                
 ionalEmbedding)                                                                                  
                                                                                                  
 positional_embedding_12 (Posit  (None, None, 256)   3845120     ['spanish[0][0]']          

In [52]:
import numpy as np

spa_voca = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_voca)), spa_voca))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
	tokenized_input_sentence = source_vectorization([input_sentence])
	decoded_sentence = "[start]"
	for i in range(max_decoded_sentence_length):
		tokenized_target_sentence = target_vectorization(
		[decoded_sentence])[:, :-1]
		predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
		sampled_token_index = np.argmax(predictions[0, i, :])

		# 다음 토큰 예측 문자열로 바꾸고 생성된 문장에 추가
		sampled_token = spa_index_lookup[sampled_token_index]
		decoded_sentence += " " + sampled_token
		if sampled_token == "[end]":
			break
	return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
	input_sentence = random.choice(test_eng_texts)
	print("-")
	print(input_sentence)
	print(decode_sequence(input_sentence))

-
We saw another ship far ahead.
[start] vi a otra alta le lejos [end]
-
Who do you want to speak to?
[start] a quién quieres hablar [end]
-
Why does everybody love cats?
[start] por qué todos ellos pueden los gatos [end]
-
I don't even want to be here now.
[start] no quiero ni siquiera aquí ahora [end]
-
He writes Arabic.
[start] Él escribe escribe [end]
-
I won't forget what you did.
[start] no te [UNK] lo que te [UNK] [end]
-
The trouble with him is that he is lazy.
[start] la problemas con él es que él es el ruido [end]
-
That scandal cost him his reputation.
[start] esa oportunidad le costó su cuenta [end]
-
Do you mind if I smoke here?
[start] te importa si aquí me gusta aquí [end]
-
Tom enlisted in the Army.
[start] tom se [UNK] en el ejército [end]
-
A drunk man fell down the stairs.
[start] un [UNK] [UNK] se cayó de las escaleras [end]
-
Tom doesn't feel like studying.
[start] a tom no tiene ganas de estudiar [end]
-
Physics is my favorite subject.
[start] la solución es mi de